# Day 22: Slam Shuffle

https://adventofcode.com/2019/day/22#part2

## Part 1

In [225]:
import numpy as np

In [236]:
size = 10
deck = np.arange(size).reshape(size)
print(deck)

[0 1 2 3 4 5 6 7 8 9]


In [473]:
# 1. deal into new stack
def dealNew(deck):
    newdeck = deck[::-1] 
    return newdeck

print(dealNew(deck))

[9 8 7 6 5 4 3 2 1 0]


In [474]:
# 2. cut N cards (N can be negative)
def cutN(deck,N):
    if N>0:
        cut = deck[0:N]
        left = deck[N:]
        newdeck = np.append(left,cut)
    elif N<0:
        cut = deck[len(deck)+N:]
        left = deck[:N]
        newdeck = np.append(cut,left)
    return newdeck
    
print(cutN(deck,3))

[3 4 5 6 7 8 9 0 1 2]


In [475]:
# 3. deal with increment N
def dealWithIncrementN(deck,N):
    newdeck = np.zeros(len(deck))
    j = 0
    for i in range(len(deck)):
        #print(j)
        newdeck[j] = deck[i]
        j = (j+N)%len(deck)
    return newdeck
 
deck = np.arange(size).reshape(size)
print(dealWithIncrementN(deck,3))

[    0.  3336.  6672. ... 10006.  3335.  6671.]


In [476]:
lines = [
'deal with increment 7',
'deal into new stack',
'deal into new stack']
# Result: 0 3 6 9 2 5 8 1 4 7 --> OK

lines = [
'cut 6',
'deal with increment 7',
'deal into new stack' ] 
#Result: 3 0 7 4 1 8 5 2 9 6 --> OK

lines = [
'deal with increment 7',
'deal with increment 9',
'cut -2' ]
#Result: 6 3 0 7 4 1 8 5 2 9 --> OK

lines = [
'deal into new stack',
'cut -2',
'deal with increment 7',
'cut 8',
'cut -4',
'deal with increment 7',
'cut 3',
'deal with increment 9',
'deal with increment 3',
'cut -1' ]
# Result: 9 2 5 8 1 4 7 0 3 6 --> OK

size = 10

In [477]:
with open("input22.txt") as f:
    lines = [l.rstrip('\n') for l in f]
#lines

size = 10007

In [478]:
s1 = 'deal into new stack'
s2 = 'cut '
s3 = 'deal with increment '

deck = np.arange(size).reshape(size)

for l in lines:
    if l == s1:
        deck = dealNew(deck)
    elif l[0:4]== s2:
        n = l.split(' ')
        N = int(n[1])
        deck = cutN(deck,N)
    else:
        n = l.split(' ')
        N = int(n[3])
        deck = dealWithIncrementN(deck,N)

In [479]:
card = 2019
pos = np.where(deck==card)[0][0]
print("Card",card,"is found at position",pos)

Card 2019 is found at position 7171


## Part 2

(in italiano qualche suggerimento matematico di Zar da http://www.frenf.it/earlyadopters/p/marcodelmastro/1006619 )

L'idea sarebbe quella di trasformare ogni tipo di ridistribuzione di carte in una funzione lineare. Per esempio, il deal diventa questo calcolo: return (N-x-1) % N, il cut diventa (x-n) % N e il deal with increment diventa (x*n) % N. A questo punto la sequenza di istruzioni diventa una composizione di funzioni lineari, quindi alla fine tutto il procedimento si può riassumere in una operazione y = ax+b.

In [513]:
with open("input22.txt") as f:
    lines = [l.rstrip('\n') for l in f]

s1 = 'deal into new stack'
s2 = 'cut '
s3 = 'deal with increment '

seq = []

for l in lines:
    if l == s1:
        seq.append([1,0])
    elif l[0:4]== s2:
        ll = l.split(' ')
        n = int(ll[1])
        seq.append([2,n])
    else:
        ll = l.split(' ')
        n = int(ll[3])
        seq.append([3,n])

In [514]:
# mod operation is factorizable!

def deal(x,N):
    return (N-x-1) #% N

def cut(x,N,n):
    return (x-n) #% N

def dealwi(x,N,n):
    return (x*n) #% N

def sequence(x,N,seq):
    for s in seq:
        if s[0]==1:
            x = deal(x,N)
        elif s[0]==2:
            x = cut(x,N,s[1])
        elif s[0]==3:
            x = dealwi(x,N,s[1])
    return x % N

In [515]:
x = 2019
N = 10007
print(sequence(x,N,seq))

7171


`sequence()` solves Part 1 implementing all instructions as operations on a single card/value, and that's good.

On the other hand, it's still defined as a sequence of operations, while I'd like to **condensate it in a simplified y=ax+b form**. Since each operation is a linear tranformation (plus a `mod()` operation that is factorisable), I can compute the "cumulative" factors a and b factor as "sum" of the coefficients of all transformations (a's needs to be multipled, b's are summed, but I should not forget to also multiply b's if current tranformation has a scaling factor!).

In [588]:
def linearSeq(seq,N):
    a = 1
    b = 0
    for s in seq:
        if s[0]==1:
            a *= -1
            b = (-1*b)+N-1 # b needs to be multiplied by -1 (a of current stranforamtion) before shifting
        elif s[0]==2:
            #a *= 1
            b -= s[1]
        elif s[0]==3:
            a *= s[1]
            b *= s[1]
    return a%N,b%N

a, b = linearSeq(seq,N)
print("a =",a,"b =",b)

a = 5636 b = 6046


In [589]:
x = 2019
y = (a * x + b)%N
print("x =",x,"--> y =",y)

x = 2019 --> y = 7171


Per risolvere la seconda parte del quesito serve la funzione inversa, cioè x = (y-b)/a, dove però non si può davvero dividere per a perché stiamo lavorando con numeri interi, quindi la divisione per a deve diventare una moltiplicazione per l'inverso moltiplicativo di a.

Per trovare la funzione inversa, posso usare l'algoritmo di Euclide esteso: http://www.di-srv.unisa.it/~ads/ads/Sicurezza_files/NumberTheory.pdf


In [590]:
def EuclidGDC(a,n):
    '''Euclid algorithm to computed GCD'''
    if n==0:
        return a
    else:
        return EuclidGDC(n, a%n)

def ExtendedEuclidGDC(a,n): 
    '''Extended Euclid algorithm to computed GCD and integer pair (x,y) that satifies d = gcd(a,n) = a*x+n*y'''
    if n==0:
        return a,1,0
    di, xi, yi = ExtendedEuclidGDC(n,a%n)
    d = di
    x = yi
    y = xi - a//n * yi # use // for integer division!
    return d, x, y 
    

d,x,y = ExtendedEuclidGDC(99,78)
d,x,y
#=-11*99+14*78

(3, -11, 14)

I can solve an equation in the form $ax = n\mod n$ it with Euclid. Solutions exists if and only if $g|b$ where $g = \gcd(a,n)$. In this case I have $g$ solutions:
$$ x'\frac{b}{g} + i \frac{n}{g} \quad\text{for}\quad i = 0,1,..., g-1$$

In my case I want to invert $y=ax+b \mod n$, so I want to find $a^{-1}\mod{n}$ and $b/a\mod{n}$ to be able to compute:
$$ x = y \frac{1}{a} - \frac{b}{a}\mod n$$

In [596]:
# example: compute di 5^-1 mod 7
d,x,y = ExtendedEuclidGDC(5,7)
print("5^-1 mod 7 =",x)

5^-1 mod 7 = 3


In [599]:
D,inva,Y = ExtendedEuclidGDC(a,N)
print("a =", a,"--> 1/a mod N =",inva)

a = 5636 --> 1/a mod N = 3631


In [593]:
def invSeq(y,inva,b,n):
    return (y * inva - b * inva)%n

invSeq(7171,inva,b,N)

2019

La parte 2 chiede di calcolare l'inverso di 2020 dopo aver applicato la funzione inversa 101741582076661 volte, cosa improponibile. Il giochetto è (chiamiamo G la funzione inversa): calcolo G(G(x)), che chiamo G2. Poi calcolo G2(G2(x)), che chiamo G4, poi G4(G4(x)), che chiamo G16. E così via, raddoppiando, in un numero di passi dell'ordine del logaritmo in base 2 di 101741582076661 si arriva al risultato.

Questa è l'idea: http://proooof.blogspot.com/2010/05/alice-bob-e-eva-modpow.html

In [595]:
# Summary of previous steps, to be recompute with new N (larger deck)
# N = 119315717514047
# M = 101741582076661

N = 10007
x = 2019
y = 7171

a, b = linearSeq(seq,N)
D,inva,Y = ExtendedEuclidGDC(a,N)
invSeq(y,inva,b,N)

2019